# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [2]:
class Matrix:
    def __init__(self, data, cols=None):
        if isinstance(data, int) and isinstance(cols, int):
            # Initialize an n x m zero matrix
            self.rows = data
            self.cols = cols
            self.matrix = [[0] * cols for _ in range(data)]
        elif isinstance(data, list):
            # Initialize with a list of lists
            if not all(isinstance(row, list) for row in data):
                raise ValueError("All rows must be lists")
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns")

            self.matrix = data
            self.rows = len(data)
            self.cols = len(data[0])
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            return self.matrix[i][j]
        return self.matrix[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            self.matrix[i][j] = value
        else:
            self.matrix[key] = value

    def __repr__(self):
        return '\n'.join(str(row) for row in self.matrix)


# Testing the implementation
if __name__ == "__main__":
    # Test zero matrix initialization
    m1 = Matrix(3, 3)
    print("Zero matrix initialization result:")
    print(m1)

    # Test initialization with a list of lists
    m2 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    print("\nList of lists initialization result:")
    print(m2)

    # Test invalid initialization (uneven rows)
    try:
        m3 = Matrix([[1, 2], [3, 4, 5]])
    except ValueError as e:
        print("\nCaught expected ValueError for uneven rows:", e)

    # Test indexing
    print("\nIndexing test result:")
    print(f"m2[1, 2] = {m2[1, 2]}")
    print(f"m2[2][1] = {m2[2][1]}")

    # Test setting values
    m1[1, 1] = 9
    print("\nUpdated matrix m1 after setting m1[1, 1] to 9:")
    print(m1)


Zero matrix initialization result:
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

List of lists initialization result:
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

Caught expected ValueError for uneven rows: All rows must have the same number of columns

Indexing test result:
m2[1, 2] = 6
m2[2][1] = 8

Updated matrix m1 after setting m1[1, 1] to 9:
[0, 0, 0]
[0, 9, 0]
[0, 0, 0]


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [4]:
class Matrix:
    def __init__(self, data, cols=None):
        if isinstance(data, int) and isinstance(cols, int):
            # Initialize an n x m zero matrix
            self.rows = data
            self.cols = cols
            self.matrix = [[0] * cols for _ in range(data)]
        elif isinstance(data, list):
            # Initialize with a list of lists
            if not all(isinstance(row, list) for row in data):
                raise ValueError("All rows must be lists")
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns")

            self.matrix = data
            self.rows = len(data)
            self.cols = len(data[0])
        else:
            raise ValueError("Invalid initialization parameters")

    def shape(self):
        return (self.rows, self.cols)

    def transpose(self):
        transposed_data = [[self.matrix[j][i] for j in range(self.rows)] for i in range(self.cols)]
        return Matrix(transposed_data)

    def row(self, n):
        if n < 0 or n >= self.rows:
            raise IndexError("Row index out of range")
        return Matrix([self.matrix[n]])

    def column(self, n):
        if n < 0 or n >= self.cols:
            raise IndexError("Column index out of range")
        return Matrix([[self.matrix[i][n]] for i in range(self.rows)])

    def to_list(self):
        return self.matrix

    def block(self, n_0, n_1, m_0, m_1):
        if not (0 <= n_0 < n_1 <= self.rows) or not (0 <= m_0 < m_1 <= self.cols):
            raise IndexError("Block indices out of range")
        block_data = [row[m_0:m_1] for row in self.matrix[n_0:n_1]]
        return Matrix(block_data)

    def __getitem__(self, key):
        if isinstance(key, tuple):
            if all(isinstance(k, int) for k in key):
                i, j = key
                return self.matrix[i][j]
            elif all(isinstance(k, slice) for k in key):
                row_slice, col_slice = key
                sliced_data = [row[col_slice] for row in self.matrix[row_slice]]
                return Matrix(sliced_data)
            else:
                raise TypeError("Invalid index type")
        elif isinstance(key, int):
            return self.matrix[key]
        elif isinstance(key, slice):
            sliced_data = self.matrix[key]
            return Matrix(sliced_data)
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            self.matrix[i][j] = value
        else:
            self.matrix[key] = value

    def __repr__(self):
        return '\n'.join(str(row) for row in self.matrix)


if __name__ == "__main__":
    m1 = Matrix(3, 3)
    print("Zero matrix initialization result:")
    print(m1)

    m2 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    print("\nList of lists initialization result:")
    print(m2)

    try:
        m3 = Matrix([[1, 2], [3, 4, 5]])
    except ValueError as e:
        print("\nCaught expected ValueError for uneven rows:", e)

    print("\nShape of m2:", m2.shape())

    m2_transposed = m2.transpose()
    print("\nTransposed matrix m2:")
    print(m2_transposed)

    print("\nRow 1 of m2:")
    print(m2.row(1))

    print("\nColumn 1 of m2:")
    print(m2.column(1))

    print("\nBlock of m2 (0:2, 0:2):")
    print(m2.block(0, 2, 0, 2))

    print("\nIndexing m2[1, 2]:", m2[1, 2])
    print("Indexing m2[2][1]:", m2[2][1])

    print("\nSlicing m2[0:2, 0:2]:")
    print(m2[0:2, 0:2])

    m1[1, 1] = 9
    print("\nUpdated matrix m1 after setting m1[1, 1] to 9:")
    print(m1)


Zero matrix initialization result:
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

List of lists initialization result:
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

Caught expected ValueError for uneven rows: All rows must have the same number of columns

Shape of m2: (3, 3)

Transposed matrix m2:
[1, 4, 7]
[2, 5, 8]
[3, 6, 9]

Row 1 of m2:
[4, 5, 6]

Column 1 of m2:
[2]
[5]
[8]

Block of m2 (0:2, 0:2):
[1, 2]
[4, 5]

Indexing m2[1, 2]: 6
Indexing m2[2][1]: 8

Slicing m2[0:2, 0:2]:
[1, 2]
[4, 5]

Updated matrix m1 after setting m1[1, 1] to 9:
[0, 0, 0]
[0, 9, 0]
[0, 0, 0]


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [6]:
class Matrix:
    def __init__(self, data, cols=None):
        if isinstance(data, int) and isinstance(cols, int):
            # Initialize an n x m zero matrix
            self.rows = data
            self.cols = cols
            self.matrix = [[0] * cols for _ in range(data)]
        elif isinstance(data, list):
            # Initialize with a list of lists
            if not all(isinstance(row, list) for row in data):
                raise ValueError("All rows must be lists")
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns")

            self.matrix = data
            self.rows = len(data)
            self.cols = len(data[0])
        else:
            raise ValueError("Invalid initialization parameters")

    def shape(self):
        return (self.rows, self.cols)

    def transpose(self):
        transposed_data = [[self.matrix[j][i] for j in range(self.rows)] for i in range(self.cols)]
        return Matrix(transposed_data)

    def row(self, n):
        if n < 0 or n >= self.rows:
            raise IndexError("Row index out of range")
        return Matrix([self.matrix[n]])

    def column(self, n):
        if n < 0 or n >= self.cols:
            raise IndexError("Column index out of range")
        return Matrix([[self.matrix[i][n]] for i in range(self.rows)])

    def to_list(self):
        return self.matrix

    def block(self, n_0, n_1, m_0, m_1):
        if not (0 <= n_0 < n_1 <= self.rows) or not (0 <= m_0 < m_1 <= self.cols):
            raise IndexError("Block indices out of range")
        block_data = [row[m_0:m_1] for row in self.matrix[n_0:n_1]]
        return Matrix(block_data)

    def __getitem__(self, key):
        if isinstance(key, tuple):
            if all(isinstance(k, int) for k in key):
                i, j = key
                return self.matrix[i][j]
            elif all(isinstance(k, slice) for k in key):
                row_slice, col_slice = key
                sliced_data = [row[col_slice] for row in self.matrix[row_slice]]
                return Matrix(sliced_data)
            else:
                raise TypeError("Invalid index type")
        elif isinstance(key, int):
            return self.matrix[key]
        elif isinstance(key, slice):
            sliced_data = self.matrix[key]
            return Matrix(sliced_data)
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            self.matrix[i][j] = value
        else:
            self.matrix[key] = value

    def __repr__(self):
        return '\n'.join(str(row) for row in self.matrix)


def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0.0)

def ones(n, m):
    return constant(n, m, 1.0)

def eye(n):
    data = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
    return Matrix(data)


if __name__ == "__main__":
    print("Zero matrix initialization result:")
    print(zeros(3, 3))

    print("\nOnes matrix initialization result:")
    print(ones(2, 2))

    print("\nConstant matrix initialization result:")
    print(constant(2, 3, 5))

    print("\nIdentity matrix initialization result:")
    print(eye(3))


Zero matrix initialization result:
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]

Ones matrix initialization result:
[1.0, 1.0]
[1.0, 1.0]

Constant matrix initialization result:
[5.0, 5.0, 5.0]
[5.0, 5.0, 5.0]

Identity matrix initialization result:
[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 1.0]


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [8]:
class Matrix:
    def __init__(self, data, cols=None):
        if isinstance(data, int) and isinstance(cols, int):
            # Initialize an n x m zero matrix
            self.rows = data
            self.cols = cols
            self.matrix = [[0.0] * cols for _ in range(data)]
        elif isinstance(data, list):
            # Initialize with a list of lists
            if not all(isinstance(row, list) for row in data):
                raise ValueError("All rows must be lists")
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns")
            
            self.matrix = [[float(val) for val in row] for row in data]
            self.rows = len(data)
            self.cols = len(data[0])
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            return self.matrix[i][j]
        return self.matrix[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            self.matrix[i][j] = float(value)
        else:
            self.matrix[key] = [float(val) for val in value]

    def __repr__(self):
        return '\n'.join(str(row) for row in self.matrix)

    def scalarmul(self, c):
        return Matrix([[c * self[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions to add.")
        return Matrix([[self[i, j] + N[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions to subtract.")
        return Matrix([[self[i, j] - N[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def mat_mult(self, N):
        if self.cols != N.rows:
            raise ValueError("Number of columns of M must equal number of rows of N.")
        return Matrix([
            [
                sum(self[i, k] * N[k, j] for k in range(self.cols))
                for j in range(N.cols)
            ]
            for i in range(self.rows)
        ])

    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        return Matrix([[self[i, j] * N[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def equals(self, N):
        if self.shape() != N.shape():
            return False
        return all(self[i, j] == N[i, j] for i in range(self.rows) for j in range(self.cols))

    def shape(self):
        return (self.rows, self.cols)


def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0.0)

def ones(n, m):
    return constant(n, m, 1.0)

def eye(n):
    data = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
    return Matrix(data)


if __name__ == "__main__":
    M = Matrix([[1, 2], [3, 4]])
    N = Matrix([[5, 6], [7, 8]])

    print("Scalar multiplication:")
    print(M.scalarmul(2))

    print("\nMatrix addition:")
    print(M.add(N))

    print("\nMatrix subtraction:")
    print(M.sub(N))

    print("\nMatrix multiplication:")
    print(M.mat_mult(N))

    print("\nElement-wise multiplication:")
    print(M.element_mult(N))

    print("\nMatrix equality check:")
    print(M.equals(Matrix([[1, 2], [3, 4]])))
    print(M.equals(N))


Scalar multiplication:
[2.0, 4.0]
[6.0, 8.0]

Matrix addition:
[6.0, 8.0]
[10.0, 12.0]

Matrix subtraction:
[-4.0, -4.0]
[-4.0, -4.0]

Matrix multiplication:
[19.0, 22.0]
[43.0, 50.0]

Element-wise multiplication:
[5.0, 12.0]
[21.0, 32.0]

Matrix equality check:
True
False


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [20]:
class Matrix:
    def __init__(self, data, cols=None):
        if isinstance(data, int) and isinstance(cols, int):
            # Initialize an n x m zero matrix
            self.rows = data
            self.cols = cols
            self.matrix = [[0.0] * cols for _ in range(data)]
        elif isinstance(data, list):
            # Initialize with a list of lists
            if not all(isinstance(row, list) for row in data):
                raise ValueError("All rows must be lists")
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns")
            
            self.matrix = [[float(val) for val in row] for row in data]
            self.rows = len(data)
            self.cols = len(data[0])
        else:
            raise ValueError("Invalid initialization parameters")

    def __getitem__(self, key):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            return self.matrix[i][j]
        return self.matrix[key]
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple) and len(key) == 2:
            i, j = key
            self.matrix[i][j] = float(value)
        else:
            self.matrix[key] = [float(val) for val in value]

    def __repr__(self):
        return '\n'.join(str(row) for row in self.matrix)

    def scalarmul(self, c):
        """Returns a new matrix that is the scalar product c * M."""
        return Matrix([[c * self[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def add(self, N):
        """Adds two matrices M and N."""
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions to add.")
        return Matrix([[self[i, j] + N[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def sub(self, N):
        """Subtracts matrix N from M."""
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions to subtract.")
        return Matrix([[self[i, j] - N[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def mat_mult(self, N):
        """Returns the matrix product of M and N."""
        if self.cols != N.rows:
            raise ValueError("Number of columns of M must equal number of rows of N.")
        return Matrix([
            [
                sum(self[i, k] * N[k, j] for k in range(self.cols))
                for j in range(N.cols)
            ]
            for i in range(self.rows)
        ])

    def element_mult(self, N):
        """Returns the element-wise product of M and N."""
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        return Matrix([[self[i, j] * N[i, j] for j in range(self.cols)] for i in range(self.rows)])

    def equals(self, N):
        """Checks if two matrices M and N are equal."""
        if self.shape() != N.shape():
            return False
        return all(self[i, j] == N[i, j] for i in range(self.rows) for j in range(self.cols))

    def shape(self):
        """Returns a tuple (n, m) representing the shape of the matrix."""
        return (self.rows, self.cols)

    # Overloading operators

    def __add__(self, other):
        if isinstance(other, Matrix):
            return self.add(other)
        else:
            return NotImplemented

    def __radd__(self, other):
        return self.__add__(other)

    def __sub__(self, other):
        if isinstance(other, Matrix):
            return self.sub(other)
        else:
            return NotImplemented

    def __rsub__(self, other):
        if isinstance(other, Matrix):
            return other.sub(self)
        else:
            return NotImplemented

    def __mul__(self, other):
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        elif isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            return NotImplemented

    def __rmul__(self, other):
        return self.__mul__(other)

    def __eq__(self, other):
        if isinstance(other, Matrix):
            return self.equals(other)
        else:
            return NotImplemented

    def __ne__(self, other):
        return not self.__eq__(other)


In [22]:
# Initialize matrices
M = Matrix([[1, 2], [3, 4]])
N = Matrix([[5, 6], [7, 8]])

# Scalar multiplication
result = 2 * M  # Equivalent to M * 2
print(result)

# Matrix addition
result = M + N
print(result)

# Matrix subtraction
result = M - N
print(result)

# Matrix multiplication
result = M * N
print(result)

# Equality check
is_equal = M == N
print(is_equal)


[2.0, 4.0]
[6.0, 8.0]
[6.0, 8.0]
[10.0, 12.0]
[-4.0, -4.0]
[-4.0, -4.0]
[19.0, 22.0]
[43.0, 50.0]
False


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [25]:
# Define matrices A, B, and C
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

# Compute (AB)C
AB = A.mat_mult(B)
(AB_C) = AB.mat_mult(C)

# Compute A(BC)
BC = B.mat_mult(C)
(A_BC) = A.mat_mult(BC)

# Check if (AB)C equals A(BC)
print(f"(AB)C == A(BC): {AB_C.equals(A_BC)}")


(AB)C == A(BC): True


In [27]:
# Define matrices A, B, and C
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

# Compute A(B + C)
B_plus_C = B.add(C)
A_B_plus_C = A.mat_mult(B_plus_C)

# Compute AB + AC
AB = A.mat_mult(B)
AC = A.mat_mult(C)
AB_plus_AC = AB.add(AC)

# Check if A(B + C) equals AB + AC
print(f"A(B + C) == AB + AC: {A_B_plus_C.equals(AB_plus_AC)}")


A(B + C) == AB + AC: True


In [29]:
# Define matrices A and B
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])

# Compute AB and BA
AB = A.mat_mult(B)
BA = B.mat_mult(A)

# Check if AB equals BA
print(f"AB == BA: {AB.equals(BA)}")


AB == BA: False


In [31]:
# Define matrix A
A = Matrix([[1, 2], [3, 4]])

# Define identity matrix I
I = Matrix([[1, 0], [0, 1]])

# Compute AI
AI = A.mat_mult(I)

# Check if AI equals A
print(f"AI == A: {AI.equals(A)}")


AI == A: True
